In [30]:
import rdflib
import networkx as nx
import numpy as np
%matplotlib inline

In [123]:
graph1 = nx.DiGraph()
g=rdflib.Graph()
g.load('DUL.owl')
for s,p,o in g:    
    graph1.add_edge(s, o, name=p)


graph2 = nx.DiGraph()
g=rdflib.Graph()
g.load('http://dbpedia.org/resource/Abraham_Lincoln')
for s,p,o in g:      
    graph2.add_edge(s, o, name=p)

In [127]:
set1=set(map(tuple, np.array(graph1.edges(data='name'))[:, 2:].tolist()))
set2=set(map(tuple, np.array(graph2.edges(data='name'))[:, 2:].tolist()))
print(set1.intersection(set2))

set([(u'http://www.w3.org/2000/01/rdf-schema#comment',), (u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',), (u'http://www.w3.org/2000/01/rdf-schema#label',)])


In [88]:
graph3 = nx.DiGraph()
graph3.add_edge(u='test',v='test2',name='test_prop')
graph3.add_edge(u='test18545231',v='test2',name='test_prop3')
graph3.add_edge(u='test5',v='test2',name='test_prop2')


graph4 = nx.DiGraph()
graph4.add_edge('test','test2',name='test_prop2')
graph4.add_edge('test3','test2',name='test_prop3')

print(graph3.edges(data='name'))
print(graph4.edges(data='name'))


set1=set(map(tuple, np.array(graph1.edges(data='name'))[:, 1:].tolist()))
set2=set(map(tuple, np.array(graph2.edges(data='name'))[:, 1:].tolist()))
intersection_edges2 = set1.intersection(set2)
print(intersection_edges2)
# # print(intersection_edges2)
# # print(graph3['test'])
# intersection=set()
# for edge in intersection_edges2:
#     if graph3[edge[0]][edge[1]]==graph4[edge[0]][edge[1]]:
#         intersection.add((edge, graph3[edge[0]][edge[1]]['name']))
        
# print(intersection)

[('test', 'test2', 'test_prop'), ('test5', 'test2', 'test_prop2'), ('test18545231', 'test2', 'test_prop3')]
[('test', 'test2', 'test_prop2'), ('test3', 'test2', 'test_prop3')]
set([(u'http://dbpedia.org/resource/Abraham_Lincoln', u'http://dbpedia.org/ontology/guest'), (u'http://ja.dbpedia.org/resource/\u30a8\u30a4\u30d6\u30e9\u30cf\u30e0\u30fb\u30ea\u30f3\u30ab\u30fc\u30f3', u'http://www.w3.org/2002/07/owl#sameAs'), (u'http://dbpedia.org/class/yago/WikicatRepublicanParty(UnitedStates)Politicians', u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), (u'http://dbpedia.org/resource/Abraham_Lincoln', u'http://dbpedia.org/property/spouse'), (u'http://dbpedia.org/class/yago/WikicatPresidentsOfTheUnitedStates', u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), (u'Abraham Lincoln', u'http://www.w3.org/2000/01/rdf-schema#label'), (u'http://dbpedia.org/resource/Abraham_Lincoln', u'http://dbpedia.org/ontology/author'), (u'Abraham Lincoln 1862 signature.svg', u'http://dbpedia.org/ontology/sign